In [ ]:
# Dependencies
import pandas as pd
import csv
import numpy as np

import re

from config import password
from sqlalchemy import create_engine

# CSV path
goodreads_file = "Resources/goodreads.csv"

In [ ]:
# Read CSV
goodreads_df = pd.read_csv(goodreads_file)
goodreads_df.head()

In [ ]:
# Remove unwanted columns
goodreads_df.drop(columns=['isbn', 'isbn13', '# num_pages', 'extra'], inplace=True)

# Rename columns
goodreads_df.columns = ['book_id', 'title', 'authors', 'avg_rating', 
                        'language_code', 'ratings_count', 'text_reviews_count']

# Set index to book_id
goodreads_df.set_index('book_id', inplace=True)

goodreads_df.head()

In [ ]:
# Condition to check if all rows under avg_rating column are numeric
condition = [bool(re.match("[0-9]{1}\.[0-9]{2}",x)) for x in goodreads_df["avg_rating"].to_list()]

In [ ]:
# Apply the condition to dataframe
goodreads_transformed=goodreads_df[condition]

In [ ]:
# Create Postgres engine
engine = create_engine('postgresql://postgres:'+ password + '@localhost:5432/books_db')

In [ ]:
# Read tables
engine.table_names()

In [ ]:
# Insert data into postgres table
goodreads_transformed.to_sql(name='goodreads', con=engine, if_exists='append', index=True)